## An example workflow for D3D

A D3D workflow using local ECMWF data.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon

In [ ]:
cpath = pyPoseidon.__path__[0].split('/lib/')[0] # get the current kernel path

In [ ]:
os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
import pyPoseidon.model as pmodel
import datetime
from pyPoseidon.utils import data,point
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os 

In [ ]:
pyPoseidon.__version__

In [ ]:
import numpy as np
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
from hvplot import pandas,xarray

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("dark_background")

## Choose data path

In [ ]:
# Set the env PATH for D3D's /path_to_folder_bin/lnx64/
os.environ['D3D'] = '/Users/brey/DELFT3D/SVN/7545/bin/lnx64/'  

In [ ]:
DATA_PATH = os.path.dirname(pyPoseidon.__file__)   # get path to the test data installed with the package
DATA_PATH

..or..

In [ ]:
DATA_PATH = os.getcwd() 
DATA_PATH  = os.path.abspath(os.path.join(DATA_PATH, os.pardir)) + '/pyPoseidon/'
DATA_PATH

In [ ]:
PWD = os.getcwd()
PWD

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'geometry':{'lon_min' : -25,'lon_max' : -13.,'lat_min' : 61.,'lat_max' : 69.},
     'solver':'d3d',
     'tag':'iceland',
     'resolution':0.1, #grid resoltuion 
     'map_step':60, # step for output of map field in d3d
     'restart_step': 60*12, # step for restart file
     'parameters':{'Dt':1.0, # timestep
                  } ,  
     'start_date':'2017-10-1',
     'time_frame':'12H',
     'meteo_source' : [DATA_PATH + '/tests/data/erai.grib'], #meteo files
     'engine' : 'cfgrib',
     'dem_source' : DATA_PATH + '/tests/data/dem.nc', #path for dem file
     'ncores': 4 , #number of cores
     'rpath':'./test/d3d/', #location of calc folder     
     'update':['all'] # optional list of components to be updated
    }

## Initialize

In [ ]:
#initialize a model
b = pmodel(**dic)

### set it up

In [ ]:
b.create() #set it up 

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

### execute

In [ ]:
#execute
b.run()

## ... or simply in one step


In [ ]:
#initialize a model
b = pmodel(**dic)
b.execute()

## Analysis of output 

In [ ]:
otp = data.data(**dic)#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.dem.bathymetry.hvplot('longitude','latitude',width=600,cmap='viridis')

### Visualize

In [ ]:
s1 = otp.Dataset.S1.isel(M=slice(1,-1)).isel(N=slice(1,-1))
ww = np.broadcast_to(otp.w.T == False, s1.shape )
s1 = s1.where(ww == True)
s1.hvplot.quadmesh(x='XZ', y='YZ', groupby='time', width=800,cmap='viridis', dynamic=True, rasterize=True)#, clim=(-1., 1.))#, geo=True) #* gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
s1.hvplot.quadmesh(x='XZ',y='YZ',width=800, height=600,cmap='viridis', geo=True, dynamic=True, rasterize=True) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

### Animations

In [ ]:
otp.frames(['S1'],title='SSH',vmin=-.5, vmax=.5, nv=10)

In [ ]:
otp.frames(['U1','V1'],title='Vel')#,scale=1.)

## Observation points / Validation

In [ ]:
otp.obs.locations

In [ ]:
# Get with index number
p = otp.obs.iloc(0)
print (p.head())
print (p.tail())

In [ ]:
# or get with Station Name  
p = otp.obs.loc('Iceland - Reykjavik')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
plat,plon = otp.obs.locations.loc[0,['lat','lon']]
plat, plon

In [ ]:
ts = point.point(solver = 'd3d', lon=plon,lat=plat,dataset=otp, var='S1')
ts.time_series.head()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.time_series.S1.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))

.. or ..

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.time_series.S1.hvplot(label='d3d',color='red')

### Station points